<a href="https://colab.research.google.com/github/Dangandy/covid/blob/world-map/eda/Covid_Api_Pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Current implementation of database does not have ICAN names and this is needed to work with topoJSON

Goal: use new api from [here](https://github.com/mathdroid/covid-19-api)



# Install libraries

In [1]:
pip install flask_sqlalchemy

# Imports

In [2]:
# db model
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# create & update
from datetime import datetime, timedelta
import requests

# lstm
import pandas as pd
import numpy as np
import random as rn

# model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import MeanSquaredLogarithmicError
from sklearn.model_selection import train_test_split

# predict
from keras.models import load_model
import collections

Using TensorFlow backend.


# Create Model

In [0]:
# load flask
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///site.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

# load db
db = SQLAlchemy(app)

# db models
class Stat(db.Model):
    id = db.Column(db.String(120), primary_key=True)
    country = db.Column(db.String(100), unique=False, nullable=False)
    ican3 = db.Column(db.String(3), nullable=False)
    date = db.Column(db.Date, nullable=False)
    confirmed = db.Column(db.Integer)
    deaths = db.Column(db.Integer)
    recovered = db.Column(db.Integer)
    confirmed_pred = db.Column(db.Integer)
    deaths_pred = db.Column(db.Integer)
    recovered_pred = db.Column(db.Integer)

    def __repr__(self):
        return f"Stat('{self.country}', '{self.date}', '{self.confirmed}', '{self.deaths}', '{self.recovered}',  '{self.confirmed_pred}', '{self.deaths_pred}', '{self.recovered_pred}')"


# Create Database Model

## Pseudocode

1. get countries
2. for each country, get stats for every day
3. save into database

In [0]:
# variables
base = 'https://covid19.mathdro.id'
country_url = '/api/countries'

In [54]:
# get countries
country_res = requests.get(f'{base}{country_url}')
countries = country_res.json()['countries'] ; countries[:5]

[{'iso2': 'AF', 'iso3': 'AFG', 'name': 'Afghanistan'},
 {'iso2': 'AL', 'iso3': 'ALB', 'name': 'Albania'},
 {'iso2': 'DZ', 'iso3': 'DZA', 'name': 'Algeria'},
 {'iso2': 'AD', 'iso3': 'AND', 'name': 'Andorra'},
 {'iso2': 'AO', 'iso3': 'AGO', 'name': 'Angola'}]

In [0]:
country_map = {}
for country in countries:
    # note iso can be None
    iso3, name = country.get('iso3', None), country['name']
    country_map[name] = iso3
    # print(iso3, name)
    # print(country_map)
    # break

In [56]:
len(country_map)

185

# For each country, find their ICAN CODE

In [0]:
match_country = [('Afghanistan'), ('Albania'), ('Algeria'), ('Andorra'), ('Angola'), ('Antigua and Barbuda'), ('Argentina'), ('Armenia'), ('Australia'), ('Austria'), ('Azerbaijan'), ('Bahamas'), ('Bahrain'), ('Bangladesh'), ('Barbados'), ('Belarus'), ('Belgium'), ('Benin'), ('Bhutan'), ('Bolivia'), ('Bosnia and Herzegovina'), ('Brazil'), ('Brunei'), ('Bulgaria'), ('Burkina Faso'), ('Cabo Verde'), ('Cambodia'), ('Cameroon'), ('Canada'), ('Central African Republic'), ('Chad'), ('Chile'), ('China'), ('Colombia'), ('Congo (Brazzaville)'), ('Congo (Kinshasa)'), ('Costa Rica'), ("Cote d'Ivoire"), ('Croatia'), ('Diamond Princess'), ('Cuba'), ('Cyprus'), ('Czechia'), ('Denmark'), ('Djibouti'), ('Dominican Republic'), ('Ecuador'), ('Egypt'), ('El Salvador'), ('Equatorial Guinea'), ('Eritrea'), ('Estonia'), ('Eswatini'), ('Ethiopia'), ('Fiji'), ('Finland'), ('France'), ('Gabon'), ('Gambia'), ('Georgia'), ('Germany'), ('Ghana'), ('Greece'), ('Guatemala'), ('Guinea'), ('Guyana'), ('Haiti'), ('Holy See'), ('Honduras'), ('Hungary'), ('Iceland'), ('India'), ('Indonesia'), ('Iran'), ('Iraq'), ('Ireland'), ('Israel'), ('Italy'), ('Jamaica'), ('Japan'), ('Jordan'), ('Kazakhstan'), ('Kenya'), ('Korea, South'), ('Kuwait'), ('Kyrgyzstan'), ('Latvia'), ('Lebanon'), ('Liberia'), ('Liechtenstein'), ('Lithuania'), ('Luxembourg'), ('Madagascar'), ('Malaysia'), ('Maldives'), ('Malta'), ('Mauritania'), ('Mauritius'), ('Mexico'), ('Moldova'), ('Monaco'), ('Mongolia'), ('Montenegro'), ('Morocco'), ('Namibia'), ('Nepal'), ('Netherlands'), ('New Zealand'), ('Nicaragua'), ('Niger'), ('Nigeria'), ('North Macedonia'), ('Norway'), ('Oman'), ('Pakistan'), ('Panama'), ('Papua New Guinea'), ('Paraguay'), ('Peru'), ('Philippines'), ('Poland'), ('Portugal'), ('Qatar'), ('Romania'), ('Russia'), ('Rwanda'), ('Saint Lucia'), ('Saint Vincent and the Grenadines'), ('San Marino'), ('Saudi Arabia'), ('Senegal'), ('Serbia'), ('Seychelles'), ('Singapore'), ('Slovakia'), ('Slovenia'), ('Somalia'), ('South Africa'), ('Spain'), ('Sri Lanka'), ('Sudan'), ('Suriname'), ('Sweden'), ('Switzerland'), ('Taiwan*'), ('Tanzania'), ('Thailand'), ('Togo'), ('Trinidad and Tobago'), ('Tunisia'), ('Turkey'), ('Uganda'), ('Ukraine'), ('United Arab Emirates'), ('United Kingdom'), ('Uruguay'), ('US'), ('Uzbekistan'), ('Venezuela'), ('Vietnam'), ('Zambia'), ('Zimbabwe'), ('Dominica'), ('Grenada'), ('Mozambique'), ('Syria'), ('Timor-Leste'), ('Belize'), ('Laos'), ('Libya'), ('West Bank and Gaza'), ('Guinea-Bissau'), ('Mali'), ('Saint Kitts and Nevis'), ('Kosovo'), ('Burma'), ('MS Zaandam'), ('Botswana'), ('Burundi'), ('Sierra Leone'), ('Malawi'), ('South Sudan'), ('Western Sahara'), ('Sao Tome and Principe'), ('Yemen')]

In [0]:
for country in match_country:
    if country not in country_map:
        print(country)

In [59]:
'Korea, South' in country_map

True

In [60]:
country_map['Korea, South']

'KOR'

In [61]:
country_map

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'Andorra': 'AND',
 'Angola': 'AGO',
 'Antigua and Barbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas': 'BHS',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bhutan': 'BTN',
 'Bolivia': 'BOL',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BWA',
 'Brazil': 'BRA',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Burkina Faso': 'BFA',
 'Burma': None,
 'Burundi': 'BDI',
 'Cabo Verde': None,
 'Cambodia': 'KHM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Central African Republic': 'CAF',
 'Chad': 'TCD',
 'Chile': 'CHL',
 'China': 'CHN',
 'Colombia': 'COL',
 'Congo (Brazzaville)': None,
 'Congo (Kinshasa)': None,
 'Costa Rica': 'CRI',
 "Cote d'Ivoire": None,
 'Croatia': 'HRV',
 'Cuba': 'CUB',
 'Cyprus': 'CYP',
 'Czechia': 'CZE',
 'Denmark': 'DNK',
 'Diamond Princess': None,
 'Dj